In [31]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras import Input

In [32]:
import zipfile
zip_file=zipfile.ZipFile('homer_bart.zip','r')
zip_file.extractall('/content')
zip_file

<zipfile.ZipFile filename='homer_bart.zip' mode='r'>

In [33]:
batch_size = 32
img_height = 64
img_width = 64
data_dir = '/content/images'

In [34]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(img_height, img_width),
    label_mode="binary",
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

Found 269 files belonging to 2 classes.


In [35]:
dataset = tf.keras.preprocessing.image_dataset_from_directory("/content/images",image_size=(64, 64),label_mode = "binary")

Found 269 files belonging to 2 classes.


In [36]:
total_batches = tf.data.experimental.cardinality(dataset).numpy()
train_batches = int(0.9 * total_batches)
test_batches = total_batches - train_batches

In [37]:
train_data = dataset.take(train_batches)
test_data = dataset.skip(train_batches)
train_data = train_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data = test_data.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [38]:
preprocess = tf.keras.Sequential(
    [tf.keras.layers.Rescaling(1./255)] # rescaling factor
)
model = tf.keras.Sequential()
model.add(Input((64,64,3)))
model.add(Flatten())
model.add(preprocess)
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [39]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [40]:
model.fit(train_data,
          epochs = 40,
          batch_size = 32,
          verbose=1,
          validation_data = test_data)

Epoch 1/40
8/8 [==============================] - 2s 48ms/step - loss: 1.9390 - accuracy: 0.5430 - val_loss: 1.0356 - val_accuracy: 0.4615
Epoch 2/40
8/8 [==============================] - 0s 5ms/step - loss: 1.4940 - accuracy: 0.4805 - val_loss: 0.5682 - val_accuracy: 0.7692
Epoch 3/40
8/8 [==============================] - 0s 7ms/step - loss: 0.8662 - accuracy: 0.6211 - val_loss: 1.1402 - val_accuracy: 0.5385
Epoch 4/40
8/8 [==============================] - 0s 5ms/step - loss: 0.4961 - accuracy: 0.7695 - val_loss: 0.9215 - val_accuracy: 0.6923
Epoch 5/40
8/8 [==============================] - 0s 5ms/step - loss: 0.4434 - accuracy: 0.7891 - val_loss: 0.7296 - val_accuracy: 0.7692
Epoch 6/40
8/8 [==============================] - 0s 5ms/step - loss: 0.4173 - accuracy: 0.7930 - val_loss: 0.7311 - val_accuracy: 0.7692
Epoch 7/40
8/8 [==============================] - 0s 5ms/step - loss: 0.4005 - accuracy: 0.7812 - val_loss: 0.4251 - val_accuracy: 0.8462
Epoch 8/40
8/8 [=================

In [41]:
loss, accuracy = model.evaluate(test_data)
print(f'Test Accuracy: {accuracy:.4f}')

1/1 [==============================] - 0s 9ms/step - loss: 0.2459 - accuracy: 0.9231
Test Accuracy: 0.9231
